In [ ]:
from config import gkey
import pandas as pd
import requests
import json
import numpy as np
from time import sleep


def getplace(lat, lon):
    """Returns the country for a given latitude and longitude"""
    # 50 requests per second limit
    sleep(0.02)
    
    # build url
    url = "https://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    
    params = {'key': gkey}
    
    # send request
    r = requests.get(url, params=params)
    j = r.json()
    # locate country
    components = j['results'][0]['address_components']
    short = None
    for c in components:
        if "country" in c['types']:
            short = c['short_name']
    return short



In [ ]:
# File paths
default = 'Raw_Data/default_features_1059_tracks.txt'
chromatic = 'Raw_Data/default_plus_chromatic_features_1059_tracks.txt'

default_clean = 'Cleaned_Data/default.csv'
chromatic_clean = 'Cleaned_Data/chromatic.csv'
country_regions_clean = 'Cleaned_Data/country_regions.csv'

In [ ]:
# read country_regions data in
country_regions = pd.read_csv(country_regions_clean)

# read in raw text file
default_df = pd.read_csv(default, header=None)
chromatic_df = pd.read_csv(chromatic, header=None)

# add lat long names
default_df.rename(columns={68: 'Latitude', 69: 'Longitude'}, inplace=True)
chromatic_df.rename(columns={116: 'Latitude', 117: 'Longitude'}, inplace=True)

In [ ]:
# Verify that latitude and longitude columns are the same
if chromatic_df['Latitude'].equals(default_df['Latitude']) and chromatic_df['Longitude'].equals(default_df['Longitude']):
    print('Lat/Long columns are identical for both dataframes')
else:
    print('Error, check columns')

In [ ]:
country_name = []
short_name = []
sub_region = []
region = []

# using getplace, create list of countries and their abbreviated names
for index, row in default_df.iterrows():
    short = getplace(row['Latitude'], row['Longitude'])
    
    # find corresponding country details using the shortname
    idx = country_regions.loc[country_regions['Short_Name'] == short]
    
    short_name.append(short)
    country_name.append(idx['Country'].values[0])
    sub_region.append(idx['Sub_Region'].values[0])
    region.append(idx['Region'].values[0])

In [ ]:
# create columns for country/region details
default_df['Short_Name'] = short_name
default_df['Country'] = country_name
default_df['Region'] = region
default_df['Sub_Region'] = sub_region

In [ ]:
# share columns with chromatic_df
chromatic_df['Short_Name'] = default_df['Short_Name']
chromatic_df['Country'] = default_df['Country']
chromatic_df['Region'] = default_df['Region']
chromatic_df['Sub_Region'] = default_df['Sub_Region']

In [ ]:
# save csvs
default_df.to_csv(default_clean, index=False)
chromatic_df.to_csv(chromatic_clean, index=False)